In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
data = pd.read_csv("data.csv", index_col=0)

data = data[['GDLCODE', 'year', 'iwipov50_real', 'transport']]

data = data.sort_values(by=['GDLCODE', 'year']).reset_index()
data.drop(columns=['index'], inplace=True)

data.head()

,GDLCODE,year,iwipov50_real,transport
0,AFGr101,1998,NaN,NaN
1,AFGr101,1999,NaN,NaN
2,AFGr101,2000,NaN,0.0
3,AFGr101,2001,NaN,0.0
4,AFGr101,2002,NaN,0.0


In [3]:
filenames = ["analysis2.0/GDL-Mean-years-education-of-adults-aged-20+-data.csv", "analysis2.0/GDL-%-population-in-urban-areas-data.csv"]
filetitle = ["educ20", "urban"]
files_dict = dict(zip(filenames, filetitle))

li = []
for filename in files_dict:
    df = pd.melt(pd.read_csv(filename, skipinitialspace=True),id_vars=['Country', 'ISO_Code', 'Level', 'GDLCODE', 'Region'],var_name='year', value_name=files_dict.get(filename))
    df.drop(columns=['Country', 'Region', 'Level'], inplace=True)
    li.append(df)
    pass
gdl = reduce(lambda df1, df2: df1.merge(df2, how="outer"), li)

gdl['year'] = gdl['year'].astype(int)

gdl.head()

,ISO_Code,GDLCODE,year,educ20,urban
0,AFG,AFGr101,1991,NaN,NaN
1,AFG,AFGr102,1991,NaN,NaN
2,AFG,AFGr103,1991,NaN,NaN
3,AFG,AFGr104,1991,NaN,NaN
4,AFG,AFGr105,1991,NaN,NaN


In [4]:
data = pd.merge(data, gdl, how="left", on=['GDLCODE', 'year'])
data

,GDLCODE,year,iwipov50_real,transport,ISO_Code,educ20,urban
0,AFGr101,1998,NaN,NaN,AFG,NaN,NaN
1,AFGr101,1999,NaN,NaN,AFG,NaN,NaN
2,AFGr101,2000,NaN,0.0,AFG,NaN,NaN
3,AFGr101,2001,NaN,0.0,AFG,NaN,NaN
4,AFGr101,2002,NaN,0.0,AFG,NaN,NaN
...,...,...,...,...,...,...,...
25237,ZWEr110,2014,NaN,0.0,ZWE,NaN,NaN
25238,ZWEr110,2015,0.0563,NaN,ZWE,10.6,100.0
25239,ZWEr110,2016,NaN,NaN,ZWE,NaN,NaN
25240,ZWEr110,2017,NaN,NaN,ZWE,NaN,NaN


In [5]:
trans_2y = data.groupby('GDLCODE', as_index=False)['transport'].rolling(2, min_periods=1).sum().shift()
data["trans2y"] = trans_2y.reset_index(level=0, drop=True)
trans_3y = data.groupby('GDLCODE', as_index=False)['transport'].rolling(3, min_periods=1).sum().shift()
data["trans3y"] = trans_3y.reset_index(level=0, drop=True)
trans_4y = data.groupby('GDLCODE', as_index=False)['transport'].rolling(4, min_periods=1).sum().shift()
data["trans4y"] = trans_4y.reset_index(level=0, drop=True)
trans_5y = data.groupby('GDLCODE', as_index=False)['transport'].rolling(5, min_periods=1).sum().shift()
data["trans5y"] = trans_5y.reset_index(level=0, drop=True)

data

,GDLCODE,year,iwipov50_real,transport,ISO_Code,educ20,urban,trans2y,trans3y,trans4y,trans5y
0,AFGr101,1998,NaN,NaN,AFG,NaN,NaN,NaN,NaN,NaN,NaN
1,AFGr101,1999,NaN,NaN,AFG,NaN,NaN,NaN,NaN,NaN,NaN
2,AFGr101,2000,NaN,0.0,AFG,NaN,NaN,NaN,NaN,NaN,NaN
3,AFGr101,2001,NaN,0.0,AFG,NaN,NaN,0.0,0.0,0.0,0.0
4,AFGr101,2002,NaN,0.0,AFG,NaN,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
25237,ZWEr110,2014,NaN,0.0,ZWE,NaN,NaN,0.0,0.0,0.0,0.0
25238,ZWEr110,2015,0.0563,NaN,ZWE,10.6,100.0,0.0,0.0,0.0,0.0
25239,ZWEr110,2016,NaN,NaN,ZWE,NaN,NaN,0.0,0.0,0.0,0.0
25240,ZWEr110,2017,NaN,NaN,ZWE,NaN,NaN,NaN,0.0,0.0,0.0


In [6]:
data = data[data['iwipov50_real'].notna()].copy().reset_index()

prev_iwipov50 = data.groupby('GDLCODE', as_index=False)['iwipov50_real'].shift()
data["prev_iwipov50"] = prev_iwipov50.reset_index(level=0, drop=True)

prev_educ20 = data.groupby('GDLCODE', as_index=False)['educ20'].shift()
data["prev_educ20"] = prev_educ20.reset_index(level=0, drop=True)

prev_urban = data.groupby('GDLCODE', as_index=False)['urban'].shift()
data["prev_urban"] = prev_urban.reset_index(level=0, drop=True)

In [7]:
prev_year = data.groupby('GDLCODE', as_index=False)['year'].shift()
data["prev_year"] = prev_year.reset_index(level=0, drop=True)

data['prev_year'] = data['prev_year'].fillna(0.0).astype(int)

data = data[data['prev_iwipov50'].notna()].copy().reset_index()
data.drop(columns=['index', 'level_0'], inplace=True)

In [8]:
data = data[data['trans3y'].notna()].copy().reset_index()
data.head()

,index,GDLCODE,year,iwipov50_real,transport,ISO_Code,educ20,urban,trans2y,trans3y,trans4y,trans5y,prev_iwipov50,prev_educ20,prev_urban,prev_year
0,0,AFGr101,2015,0.368,NaN,AFG,4.81,52.50,0.0,0.0,0.0,0.0,0.519,4.30,47.90,2010
1,1,AFGr102,2015,0.765,NaN,AFG,2.64,4.93,0.0,0.0,0.0,0.0,0.831,1.90,2.60,2010
2,2,AFGr103,2015,0.779,NaN,AFG,3.29,14.60,0.0,0.0,0.0,0.0,0.788,2.58,7.22,2010
3,3,AFGr104,2015,0.376,NaN,AFG,3.33,20.50,0.0,0.0,0.0,0.0,0.778,1.59,16.00,2010
4,4,AFGr105,2015,0.616,NaN,AFG,2.75,19.70,0.0,0.0,0.0,0.0,0.710,1.58,19.60,2010


In [9]:
data['unit_of_analysis'] = data['GDLCODE'] + '_' + data['prev_year'].astype('str') + '_to_' + data['year'].astype('str')
data

,index,GDLCODE,year,iwipov50_real,transport,ISO_Code,educ20,urban,trans2y,trans3y,trans4y,trans5y,prev_iwipov50,prev_educ20,prev_urban,prev_year,unit_of_analysis
0,0,AFGr101,2015,0.3680,NaN,AFG,4.81,52.50,0.0,0.0,0.0,0.0,0.5190,4.30,47.90,2010,AFGr101_2010_to_2015
1,1,AFGr102,2015,0.7650,NaN,AFG,2.64,4.93,0.0,0.0,0.0,0.0,0.8310,1.90,2.60,2010,AFGr102_2010_to_2015
2,2,AFGr103,2015,0.7790,NaN,AFG,3.29,14.60,0.0,0.0,0.0,0.0,0.7880,2.58,7.22,2010,AFGr103_2010_to_2015
3,3,AFGr104,2015,0.3760,NaN,AFG,3.33,20.50,0.0,0.0,0.0,0.0,0.7780,1.59,16.00,2010,AFGr104_2010_to_2015
4,4,AFGr105,2015,0.6160,NaN,AFG,2.75,19.70,0.0,0.0,0.0,0.0,0.7100,1.58,19.60,2010,AFGr105_2010_to_2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,2092,ZWEr109,2011,0.2660,1.0,ZWE,10.50,100.00,0.0,0.0,0.0,0.0,0.2110,9.23,100.00,2006,ZWEr109_2006_to_2011
1958,2093,ZWEr109,2015,0.2560,NaN,ZWE,11.00,94.50,0.0,0.0,1.0,1.0,0.2660,10.50,100.00,2011,ZWEr109_2011_to_2015
1959,2094,ZWEr110,2006,0.1310,0.0,ZWE,9.18,100.00,0.0,0.0,0.0,0.0,0.2930,8.93,100.00,1999,ZWEr110_1999_to_2006
1960,2095,ZWEr110,2011,0.0679,0.0,ZWE,9.78,100.00,0.0,0.0,0.0,0.0,0.1310,9.18,100.00,2006,ZWEr110_2006_to_2011


In [10]:
data['trans2y_d'] = np.where(data['trans2y'] > 1, 1, 0)
data['trans3y_d'] = np.where(data['trans3y'] > 1, 1, 0)
data['trans4y_d'] = np.where(data['trans4y'] > 1, 1, 0)
data['trans5y_d'] = np.where(data['trans5y'] > 1, 1, 0)

In [11]:
data['change_iwipov50'] = data['iwipov50_real'] - data['prev_iwipov50']
data['change_educ20'] = data['educ20'] - data['prev_educ20']
data['change_urban'] = data['urban'] - data['prev_urban']
data['years_between_measurements'] = data['year'] - data['prev_year']

In [12]:
data.head()

,index,GDLCODE,year,iwipov50_real,transport,ISO_Code,educ20,urban,trans2y,trans3y,...,prev_year,unit_of_analysis,trans2y_d,trans3y_d,trans4y_d,trans5y_d,change_iwipov50,change_educ20,change_urban,years_between_measurements
0,0,AFGr101,2015,0.368,NaN,AFG,4.81,52.50,0.0,0.0,...,2010,AFGr101_2010_to_2015,0,0,0,0,-0.151,0.51,4.60,5
1,1,AFGr102,2015,0.765,NaN,AFG,2.64,4.93,0.0,0.0,...,2010,AFGr102_2010_to_2015,0,0,0,0,-0.066,0.74,2.33,5
2,2,AFGr103,2015,0.779,NaN,AFG,3.29,14.60,0.0,0.0,...,2010,AFGr103_2010_to_2015,0,0,0,0,-0.009,0.71,7.38,5
3,3,AFGr104,2015,0.376,NaN,AFG,3.33,20.50,0.0,0.0,...,2010,AFGr104_2010_to_2015,0,0,0,0,-0.402,1.74,4.50,5
4,4,AFGr105,2015,0.616,NaN,AFG,2.75,19.70,0.0,0.0,...,2010,AFGr105_2010_to_2015,0,0,0,0,-0.094,1.17,0.10,5


In [13]:
gdp = pd.read_csv("analysis2.0/Data_Extract_From_World_Development_Indicators/c0195b9c-a919-4e3f-a531-e01c96fb1be8_Data.csv")
gdp.drop(gdp.index[-5:], inplace=True)
gdp.drop(['Time Code', 'Country Name'], axis=1, inplace=True)
gdp.rename(columns = {"GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD]": "prev_gdppc", "Country Code": "ISO_Code", "Time": "prev_year"}, inplace = True)
gdp.replace('..', np.nan, inplace=True)
gdp['prev_year'] = gdp['prev_year'].astype(int)
gdp.head()

,prev_year,ISO_Code,prev_gdppc
0,1998,AFG,NaN
1,1998,ALB,1835.64668130129
2,1998,DZA,3414.56223981336
3,1998,ASM,NaN
4,1998,AND,38196.3937249671


In [14]:
data = pd.merge(data, gdp, how="left", on=['ISO_Code', 'prev_year'])
indexNames = data[ data['ISO_Code'] == 'TUN' ].index
data.drop(indexNames , inplace=True)
data.head()

,index,GDLCODE,year,iwipov50_real,transport,ISO_Code,educ20,urban,trans2y,trans3y,...,unit_of_analysis,trans2y_d,trans3y_d,trans4y_d,trans5y_d,change_iwipov50,change_educ20,change_urban,years_between_measurements,prev_gdppc
0,0,AFGr101,2015,0.368,NaN,AFG,4.81,52.50,0.0,0.0,...,AFGr101_2010_to_2015,0,0,0,0,-0.151,0.51,4.60,5,543.303041863931
1,1,AFGr102,2015,0.765,NaN,AFG,2.64,4.93,0.0,0.0,...,AFGr102_2010_to_2015,0,0,0,0,-0.066,0.74,2.33,5,543.303041863931
2,2,AFGr103,2015,0.779,NaN,AFG,3.29,14.60,0.0,0.0,...,AFGr103_2010_to_2015,0,0,0,0,-0.009,0.71,7.38,5,543.303041863931
3,3,AFGr104,2015,0.376,NaN,AFG,3.33,20.50,0.0,0.0,...,AFGr104_2010_to_2015,0,0,0,0,-0.402,1.74,4.50,5,543.303041863931
4,4,AFGr105,2015,0.616,NaN,AFG,2.75,19.70,0.0,0.0,...,AFGr105_2010_to_2015,0,0,0,0,-0.094,1.17,0.10,5,543.303041863931


In [15]:
data.to_csv('dataset_v2.csv')

In [16]:
#iwipov50_real = real values of iwipov50 (not inter- or extrapolated values)
#year = observed year
#prev_year = the year of the last real iwipov50 value before the observed year
#years_between_measurements
#transport = number transport project in the observed year
#trans2y/3y/4y/5y = cumulative number of transport projects committed in the 2/3/4/5 years prior to the observed year
#trans2y/3y/4y/5y_d = dummy indicating 1 or more projects committed in the 2/3/4/5 years prior to the observed year